# ResNet 16

## Configurations

### Path exploring
* Img Location
`/kaggle/input/daanforestpark/_/DaanForestPark/_/_/_.jpg`
* Label Location
`/kaggle/input/daanforestpark/label2.csv`

In [9]:
""" Show all the data path here """
""" you need to define your datapath """
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass
#         print(os.path.join(dirname, filename))

### Install and import necessary libraries
First, you need to load required package as the following so that we can start enjoying this journey.

In [10]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from PIL import Image

import os
import random
from datetime import datetime

import torch
from torch.utils.data import Dataset, DataLoader,SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

import torchvision.models as models
from torchvision import transforms
from torchsummary import summary

假設沒有套件,打開下方Console，輸入`pip install xxx` e.g., `pip install torchsummary`

#### GPU or not?
如果你本地有GPU，建議安裝cuda來做訓練。如果是在Kaggle的環境上訓練，可以開啟右側直欄的`Accelerator`，裡面有使用GPU or TPU options.

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#### Path specified
##### Where is the Data in Kaggle ?
在一開始就有找到所有data的路徑，如果不是很確定就uncomment `# print` 再run一次看看，它會顯示data的location.
右邊直欄也有`Data`可以讓你檢查資料的位置。

In [12]:
# the path of .csv file
DATA_DIR = '/kaggle/input/daanforestpark'
LABEL_DIR = os.path.join(DATA_DIR, 'label2.csv')
IMAGE_DIR = DATA_DIR # since all data were located in the same directory.

### Hyperparameters

這裡是模型需要調整的參數，盡量不要做更動。

In [13]:
NUM_WORKERS = 4

EPOCHES = 50
BATCH_SIZE = 32
PATIENCE = 5
SPLIT_VALID = .2
LR = 1e-2
L1_ratio = 1e-4
L2_ratio = 1e-3

### Augmentation 

In [14]:
# ToPILImage() -> Resize() -> ToTensor()
transform = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.3),
#         transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])        
        ])

### Define the DataSet and the DataLoader

In [15]:
class MyDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        _images, _labels = [], []
        # total amount of dataset 
        _number = 0
        # Reading the categorical file
        label_df = pd.read_csv(label_dir)
        
        # Iterate all files including .jpg inages  
        for subdir, dirs, files in tqdm(os.walk(image_dir)):
            for filename in files:
                if len(subdir.split(os.sep)) >5:
                    # 注意到這裡如果不能讀檔一定是這裡發生問題,路徑要檢查一下
                    corr_label = label_df[label_df['dirpath']==os.sep.join(subdir.split(os.sep)[5:])]['label'].values
                    if corr_label.size!= 0 and filename.endswith(('jpg')):
                        _images.append(subdir + os.sep + filename)
                        _labels.append(corr_label)
                        _number+=1
        # Randomly arrange data pairs
        mapIndexPosition = list(zip(_images, _labels))
        random.shuffle(mapIndexPosition)
        _images, _labels = zip(*mapIndexPosition)

        self._image = iter(_images)
        self._labels = iter(_labels)
        self._number = _number
        self._category = label_df['label'].nunique()
        self.transform = transform
        
    def __len__(self):
        return self._number

    def __getitem__(self, index):    
        img = next(self._image)
        lab = next(self._labels)
        
        img = self._loadimage(img)
        if self.transform:
            img = self.transform(img)        
        return img, lab
     
    def _categorical(self, label):
        return np.arange(self._category) == label[:,None]
    
    def _loadimage(self, file):
        return Image.open(file).convert('RGB')
    
    def get_categorical_nums(self):
        return self._category
    

In [16]:
train_dataset = MyDataset(IMAGE_DIR, LABEL_DIR, transform=transform)

valid_size = SPLIT_VALID
num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, sampler=train_sampler, drop_last=True)
valid_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, sampler=valid_sampler, drop_last=True)

### ResNet18

In [17]:
# model = SimpleCNN(train_dataset.get_categorical_nums()).to(device)
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, train_dataset.get_categorical_nums())

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth


In [18]:
# channels, H, W
model = model_ft.to(device=device)
summary(model, input_size=(3, 64, 64))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           9,408
       BatchNorm2d-2           [-1, 64, 64, 64]             128
              ReLU-3           [-1, 64, 64, 64]               0
         MaxPool2d-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
              ReLU-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
             ReLU-10           [-1, 64, 32, 32]               0
       BasicBlock-11           [-1, 64, 32, 32]               0
           Conv2d-12           [-1, 64, 32, 32]          36,864
      BatchNorm2d-13           [-1, 64, 32, 32]             128
             ReLU-14           [-1, 64,

In [19]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [20]:
def save_checkpoint(model, state, filename, ckptname='resNet.ckpt'):
    if not os.path.isdir(filename):
        try: 
            os.mkdir(filename) 
            print('Create {}'.format(filename))
        except OSError as err: 
            raise err

    _ckpt = os.path.join(filename, ckptname)
    torch.save(state, _ckpt)
    print('Saving the {} in {}'.format(ckptname, filename))

    """ If you wwanna save the whole model, Uncomment below """
#     _model = os.path.join(filename, 'model_best.ckpt')
#     torch.save(model, _model)

In [21]:
optimizer = torch.optim.Adam(model.parameters(), lr=LR)   # optimize all cnn parameters
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=0, verbose=True)
schedulerAcc = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.9, patience=0, verbose=True)
criterion = nn.CrossEntropyLoss().to(device=device)

# early stopping
min_val_loss = np.Inf
patience = PATIENCE
global_step = 1

# build the writer file
write_file = 'runs/experiment_{}'.format(datetime.now().strftime('%f'))
writer = SummaryWriter(write_file)
# define the ckpt path
writer_ckpt_path = os.path.join(write_file, 'ckpt')

model_ft.train()

for epoch in range(EPOCHES):
    for i, (img_batch, label_batch) in tqdm(enumerate(train_loader)):    
        optimizer.zero_grad()      
        img_batch = img_batch.to(device=device)
        label_batch = label_batch.to(device=device)  
        output = model_ft(img_batch)
        loss = criterion(output, label_batch.squeeze())
        
        # l2 Regularization loss
        l2_regularization = 0
        l1_regularization = 0
        for p in model.parameters():
            l1_regularization += torch.norm(p, 1)
            l2_regularization += torch.norm(p, 2)
#         loss = loss + (L1_ratio * l1_regularization) + (L2_ratio * l2_regularization)
        loss = loss + (L2_ratio * l2_regularization)

        loss.backward()
        # clip the grandient value for avoiding explosion
        nn.utils.clip_grad_norm_(model.parameters(), 0.9) 
        optimizer.step()

        # Compute accuracy
        _, predicted = torch.max(output.cpu().data, 1)
        accuracy = torch.sum(predicted == label_batch.cpu().data.view(-1), dtype=torch.float32) / BATCH_SIZE
        
        # Write tensorboard
        writer.add_scalar('train/Accuracy', accuracy.item(), global_step)
        writer.add_scalar('train/Loss', loss.item(), global_step)
        writer.add_scalar('train/L1RegLoss', l1_regularization.item(), global_step)
        writer.add_scalar('train/L2RegLoss', l2_regularization.item(), global_step)
        writer.add_scalar('train/LR', get_lr(optimizer), global_step)
                
        global_step += 1
        
        if i % 50== 0:
            print('epoch {}, step {}, \
            total_loss={:.3f}, \
            accuracy={:.3f}'.format(epoch+1, i, loss.item(), accuracy.item()))
    
    
    print('--- Validation phase ---')
    eval_loss = 0
    with torch.no_grad():
        for i, (img_batch, label_batch) in enumerate(valid_loader):
            output = model(img_batch.to(device))
            _, predicted = torch.max(output.cpu().data, 1)
            loss = criterion(output, label_batch.to(device).squeeze())
            accuracy = torch.sum(predicted == label_batch.data.view(-1), dtype=torch.float32) / BATCH_SIZE
            eval_loss += loss.item()
            
            # Write tensorboard
            for cat in range(train_dataset.get_categorical_nums()):
                writer.add_pr_curve('valid/pr_curve', (label_batch == cat).int().squeeze(),
                                    (predicted == cat).int().squeeze(), epoch*len(valid_loader)+i)
#             writer.add_images('valid/image_batch', img_batch, epoch*len(valid_loader)+i)
            writer.add_scalar('valid/Accuracy', accuracy.item(), epoch*len(valid_loader)+i)
            writer.add_scalar('valid/Loss', loss.item(), epoch*len(valid_loader)+i)
    
    eval_loss = eval_loss / len(valid_loader)
    
    scheduler.step(eval_loss)
    schedulerAcc.step(eval_loss)
    
    print('epoch {}, val_loss={:.3f}'.format(epoch+1, eval_loss))

    ## Early Stopping
    if eval_loss <= min_val_loss:
        
        """ file path you should define it => 'ckpt/resNet_{}.ckpt' """
        save_checkpoint(model, 
            {
            'epoch': epoch+1,
            'state_dict': model.state_dict(),
            'best_loss': eval_loss,
            'optimizer' :optimizer.state_dict(),
            }, writer_ckpt_path)
        min_val_loss = eval_loss
    else:
        patience-=1
    if patience == 0:
        print('Early stopping')
        break

writer.close()
print('Finish all training !')

epoch 1, step 0,             total_loss=30.513,             accuracy=0.000
epoch 1, step 50,             total_loss=17.082,             accuracy=0.188
epoch 1, step 100,             total_loss=15.108,             accuracy=0.000
epoch 1, step 150,             total_loss=13.228,             accuracy=0.188
epoch 1, step 200,             total_loss=13.038,             accuracy=0.188
epoch 1, step 250,             total_loss=11.348,             accuracy=0.281
epoch 1, step 300,             total_loss=9.767,             accuracy=0.125
epoch 1, step 350,             total_loss=9.822,             accuracy=0.094
epoch 1, step 400,             total_loss=8.677,             accuracy=0.188
epoch 1, step 450,             total_loss=9.051,             accuracy=0.312
epoch 1, step 500,             total_loss=8.352,             accuracy=0.219

--- Validation phase ---
epoch 1, val_loss=3.092


epoch 2, step 0,             total_loss=8.888,             accuracy=0.219
epoch 2, step 50,             total_loss=6.795,             accuracy=0.281
epoch 2, step 100,             total_loss=7.222,             accuracy=0.188
epoch 2, step 150,             total_loss=7.078,             accuracy=0.406
epoch 2, step 200,             total_loss=7.544,             accuracy=0.281
epoch 2, step 250,             total_loss=6.406,             accuracy=0.469
epoch 2, step 300,             total_loss=6.197,             accuracy=0.438
epoch 2, step 350,             total_loss=6.972,             accuracy=0.250
epoch 2, step 400,             total_loss=7.076,             accuracy=0.375
epoch 2, step 450,             total_loss=8.541,             accuracy=0.250
epoch 2, step 500,             total_loss=6.388,             accuracy=0.344

--- Validation phase ---
Epoch     2: reducing learning rate of group 0 to 9.0000e-03.
epoch 2, val_loss=2.372


epoch 3, step 0,             total_loss=6.437,             accuracy=0.281
epoch 3, step 50,             total_loss=5.619,             accuracy=0.406
epoch 3, step 100,             total_loss=6.342,             accuracy=0.312
epoch 3, step 150,             total_loss=5.729,             accuracy=0.375
epoch 3, step 200,             total_loss=5.997,             accuracy=0.250
epoch 3, step 250,             total_loss=5.841,             accuracy=0.469
epoch 3, step 300,             total_loss=5.073,             accuracy=0.531
epoch 3, step 350,             total_loss=6.033,             accuracy=0.344
epoch 3, step 400,             total_loss=5.431,             accuracy=0.375
epoch 3, step 450,             total_loss=7.974,             accuracy=0.625
epoch 3, step 500,             total_loss=5.594,             accuracy=0.438

--- Validation phase ---
Epoch     3: reducing learning rate of group 0 to 8.1000e-03.
epoch 3, val_loss=1.999


epoch 4, step 0,             total_loss=6.188,             accuracy=0.312
epoch 4, step 50,             total_loss=5.411,             accuracy=0.562
epoch 4, step 100,             total_loss=6.232,             accuracy=0.500
epoch 4, step 150,             total_loss=5.291,             accuracy=0.594
epoch 4, step 200,             total_loss=5.405,             accuracy=0.375
epoch 4, step 250,             total_loss=4.790,             accuracy=0.500
epoch 4, step 300,             total_loss=4.483,             accuracy=0.562
epoch 4, step 350,             total_loss=6.127,             accuracy=0.500
epoch 4, step 400,             total_loss=5.151,             accuracy=0.500
epoch 4, step 450,             total_loss=6.257,             accuracy=0.469
epoch 4, step 500,             total_loss=4.717,             accuracy=0.531

--- Validation phase ---
Epoch     4: reducing learning rate of group 0 to 7.2900e-03.
epoch 4, val_loss=1.726


epoch 5, step 0,             total_loss=5.539,             accuracy=0.406
epoch 5, step 50,             total_loss=5.141,             accuracy=0.500
epoch 5, step 100,             total_loss=5.241,             accuracy=0.312
epoch 5, step 150,             total_loss=4.443,             accuracy=0.625
epoch 5, step 200,             total_loss=4.580,             accuracy=0.500
epoch 5, step 250,             total_loss=5.307,             accuracy=0.562
epoch 5, step 300,             total_loss=3.910,             accuracy=0.688
epoch 5, step 350,             total_loss=5.072,             accuracy=0.438
epoch 5, step 400,             total_loss=4.795,             accuracy=0.438
epoch 5, step 450,             total_loss=6.101,             accuracy=0.594
epoch 5, step 500,             total_loss=4.754,             accuracy=0.625

--- Validation phase ---
Epoch     5: reducing learning rate of group 0 to 6.5610e-03.
epoch 5, val_loss=1.530


...

epoch 34, step 0,             total_loss=1.036,             accuracy=0.906
epoch 34, step 50,             total_loss=1.478,             accuracy=0.750
epoch 34, step 100,             total_loss=1.364,             accuracy=0.750
epoch 34, step 150,             total_loss=1.086,             accuracy=0.844
epoch 34, step 200,             total_loss=0.845,             accuracy=0.938
epoch 34, step 250,             total_loss=0.788,             accuracy=0.969
epoch 34, step 300,             total_loss=0.970,             accuracy=0.875
epoch 34, step 350,             total_loss=0.939,             accuracy=0.906
epoch 34, step 400,             total_loss=1.381,             accuracy=0.812
epoch 34, step 450,             total_loss=1.135,             accuracy=0.875
epoch 34, step 500,             total_loss=0.800,             accuracy=0.969

--- Validation phase ---
Epoch    34: reducing learning rate of group 0 to 2.2528e-04.
epoch 34, val_loss=0.424


epoch 35, step 0,             total_loss=1.154,             accuracy=0.781
epoch 35, step 50,             total_loss=1.056,             accuracy=0.812
epoch 35, step 100,             total_loss=1.190,             accuracy=0.812
epoch 35, step 150,             total_loss=1.109,             accuracy=0.844
epoch 35, step 200,             total_loss=0.867,             accuracy=0.938
epoch 35, step 250,             total_loss=0.834,             accuracy=0.969
epoch 35, step 300,             total_loss=1.026,             accuracy=0.812
epoch 35, step 350,             total_loss=0.776,             accuracy=0.969
epoch 35, step 400,             total_loss=1.130,             accuracy=0.812
epoch 35, step 450,             total_loss=1.280,             accuracy=0.812
epoch 35, step 500,             total_loss=0.884,             accuracy=0.938

--- Validation phase ---
Epoch    35: reducing learning rate of group 0 to 2.0276e-04.
Epoch    35: reducing learning rate of group 0 to 1.8248e-04.
epoch 

epoch 36, step 0,             total_loss=1.087,             accuracy=0.844
epoch 36, step 50,             total_loss=0.914,             accuracy=0.938
epoch 36, step 100,             total_loss=1.200,             accuracy=0.812
epoch 36, step 150,             total_loss=0.792,             accuracy=0.938
epoch 36, step 200,             total_loss=1.006,             accuracy=0.906
epoch 36, step 250,             total_loss=0.862,             accuracy=0.906
epoch 36, step 300,             total_loss=0.933,             accuracy=0.906
epoch 36, step 350,             total_loss=1.012,             accuracy=0.875
epoch 36, step 400,             total_loss=1.169,             accuracy=0.844
epoch 36, step 450,             total_loss=1.241,             accuracy=0.875
epoch 36, step 500,             total_loss=0.789,             accuracy=0.969

--- Validation phase ---
Epoch    36: reducing learning rate of group 0 to 1.6423e-04.
epoch 36, val_loss=0.420


epoch 37, step 0,             total_loss=1.060,             accuracy=0.812
epoch 37, step 50,             total_loss=0.974,             accuracy=0.844
epoch 37, step 100,             total_loss=1.132,             accuracy=0.906
epoch 37, step 150,             total_loss=0.984,             accuracy=0.875
epoch 37, step 200,             total_loss=0.917,             accuracy=0.875
epoch 37, step 250,             total_loss=0.850,             accuracy=0.906
epoch 37, step 300,             total_loss=1.056,             accuracy=0.875
epoch 37, step 350,             total_loss=0.868,             accuracy=0.938
epoch 37, step 400,             total_loss=1.223,             accuracy=0.812
epoch 37, step 450,             total_loss=1.071,             accuracy=0.875
epoch 37, step 500,             total_loss=1.013,             accuracy=0.875

--- Validation phase ---
Epoch    37: reducing learning rate of group 0 to 1.4781e-04.
Epoch    37: reducing learning rate of group 0 to 1.3303e-04.
epoch 

NameError: name 'writer' is not defined

### Validation Phase

In [22]:
""" 
If you want load other pretrained model, please uncomment following two lines.
Try to load your model and put parameters in neural network structure.
Or you can recover a pretrained model including parameters and the arichitecture. 
"""
# CKPT_PATH = 'model_best.ckpt' 
# model.load_state_dict(torch.load(CKPT_PATH)['state_dict'])
model.eval()
acc = 0
for i, (img_batch, label_batch) in enumerate(valid_loader):
    output = model(img_batch.to(device))
    _, predicted = torch.max(output.cpu().data, 1)
    accuracy = torch.sum(predicted == label_batch.data.view(-1), dtype=torch.float32) / BATCH_SIZE
    acc += accuracy
print('accuracy={}'.format(acc/len(valid_loader)))

accuracy=0.9087591171264648


In [23]:
# torch.save(model, 'whole_model.ckpt')